In [2]:
!pip install qiskit qiskit-nature

  Obtaining dependency information for qiskit-nature from https://files.pythonhosted.org/packages/66/16/2bf5c270457dbda2123604108f007f4b7087fa5a3913e60f5a4f75b76d92/qiskit_nature-0.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for h5py from https://files.pythonhosted.org/packages/f0/af/dfbea0c69fe725e9e77259d42f4e14eb582eb094200aaf697feb36f513d8/h5py-3.11.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.5 MB/s eta 0:00:00a 0:00:01


In [4]:
from qiskit_aer import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import ImaginaryTimeEvolution
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.circuit.library import HartreeFock, UCCSD

# Define the molecule
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.735', basis='sto3g')

# Set up the electronic structure problem
es_problem = ElectronicStructureProblem(driver)

# Convert to qubit Hamiltonian
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True)
hamiltonian = qubit_converter.convert(es_problem.second_q_ops()[0])

# Define initial state and ansatz
num_particles = (es_problem.molecule_data_transformed.num_alpha, es_problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * es_problem.molecule_data_transformed.num_molecular_orbitals

initial_state = HartreeFock(num_spin_orbitals, num_particles, qubit_converter)
ansatz = UCCSD(qubit_converter, num_particles, num_spin_orbitals, initial_state=initial_state)

# Set up quantum instance
quantum_instance = QuantumInstance(backend=Aer.get_backend('statevector_simulator'))

# Imaginary time evolution
ite = ImaginaryTimeEvolution(quantum_instance=quantum_instance, ansatz=ansatz)
result = ite.evolve(hamiltonian)

# Extract and display results
print("Ground state energy: ", result.eigenvalue.real)


ModuleNotFoundError: No module named 'qiskit_aer'